## Extended use of Huggingface's Zero-Shot Pipeline

- In this notebook, we extend the last notebook's zero-shot learning while using custom sentences and labels to classify those texts.  
- You will also see, how multi-lingual transformer models can be used to perform various tasks in many languages.

In [45]:
from transformers import pipeline

import pandas as pd

In [46]:
classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/jopanda/.pyenv/versions/3.11.3/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [47]:
"""
We create a function to display our predictions from the model in a tabular form
"""
def get_predictions_score(prediction):
    pred_labels = prediction['labels']
    pred_scores = prediction['scores']
    seq = [prediction['sequence']]
    return  pd.concat([
                pd.DataFrame(seq),
                pd.DataFrame(pred_labels),
                pd.DataFrame(pred_scores),
            ], axis=1, ignore_index=True).rename(columns={0:'Sequence',1:'Labels', 2:'Probability'}).set_index(['Sequence'])

In [65]:
sequence = "Amazon is the longest river in the world"
candidate_labels = ["geography",  "delivery"]

pred = classifier(sequence, candidate_labels)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
Amazon is the longest river in the world,geography,0.710854
NaN,delivery,0.289146


What if we change some spellings? Here we change Amazon -> amazon. It doesn't make much difference but in some cases it will. <br> Try playing with spellings and adding or removing labels

In [49]:
sequence = "amazon is the longest river in the world"
candidate_labels = ["geography",  "delivery"]

pred = classifier(sequence, candidate_labels)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
amazon is the longest river in the world,geography,0.805673
NaN,delivery,0.194327


In the example below, you'll see how good are these models in understanding the context, with a slight spelling mistake. <br> Try changing the spelling and observe the results

In [66]:
sequence = "are we going to Oktoberfest?"
candidate_labels = ["food", "Munich", "beer", "wine", "pretzel", "sausage"] ## What if you change bear (animal) -> beer (drink)

pred = classifier(sequence, candidate_labels)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
are we going to Oktoberfest?,beer,0.489584
NaN,wine,0.295735
NaN,food,0.132666
NaN,pretzel,0.033138
NaN,sausage,0.025903
NaN,Munich,0.022975


In [67]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["food", "public health", "plants", "fruits","america"]

pred = classifier(sequence, candidate_labels)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
Who are you voting for in 2020?,fruits,0.273114
NaN,plants,0.197253
NaN,public health,0.191206
NaN,america,0.181474
NaN,food,0.156952


##### The predictions are poor as the labels are not related to the sequence. But there are ways to improve upon this. We can provide related target labels for the input sequence.


In [52]:
## Think about other labels which can improve the predictions
## HINT: Labels related to your text

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [53]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

pred = classifier(sequence, candidate_labels, multi_label=True)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
Who are you voting for in 2020?,politics,0.996493
NaN,elections,0.993664
NaN,economics,0.063439
NaN,public health,0.054793


#### Sentiment Classification

Here's an example of sentiment classification: 

In [54]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

pred = classifier(sequence, candidate_labels)
get_predictions_score(pred)

,Labels,Probability
Sequence,,
I hated this movie. The acting sucked.,negative,0.995728
NaN,positive,0.004272


So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [55]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'sequence': 'I hated this movie. The acting sucked.',
  'labels': ['negative', 'positive'],
  'scores': [0.9957284331321716, 0.004271588753908873]},
 {'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it.",
  'labels': ['positive', 'negative'],
  'scores': [0.8578124046325684, 0.14218759536743164]}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [56]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'sequence': 'I hated this movie. The acting sucked.',
  'labels': ['negative', 'positive'],
  'scores': [0.9941108822822571, 0.005889112129807472]},
 {'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it.",
  'labels': ['positive', 'negative'],
  'scores': [0.9871302843093872, 0.012869675643742085]}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.

#### Zero-shot classification in more than 100 languages



Interested in using the pipeline for languages other than English? There is a cross-lingual model on top of XLM RoBERTa which you can use by passing `model='joeddav/xlm-roberta-large-xnli'` when creating the pipeline: 

In [69]:
classifier = pipeline(
    "zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli"
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [68]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli', device=0)

/Users/jopanda/.pyenv/versions/3.11.3/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaFor

You can use it with any combination of languages. For example, let's classify a Russian sentence with English candidate labels:

In [70]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048494696617126, 0.05722113326191902, 0.037929341197013855]}

Now let's do the same but with the labels in French:



In [71]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726153612136841, 0.01712857186794281, 0.010256093926727772]}

As we discussed in the last section, the default hypothesis template is the English, `This text is {}.`. If you are working strictly within one language, it may be worthwhile to translate this to the language you are working with:

In [72]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'sequence': '¿A quién vas a votar en 2020?',
 'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109592437744141, 0.05954797938466072, 0.029492780566215515]}

The model is fine-tuned on XNLI which includes 15 languages: Arabic, Bulgarian, Chinese, English, French, German, Greek, Hindi, Russian, Spanish, Swahili, Thai, Turkish, Urdu, and Vietnamese. The base model is trained on 85 more, so the model will work to some degree for any of those in the XLM RoBERTa training corpus (see the full list in appendix A of the [XLM Roberata paper](https://arxiv.org/abs/1911.02116)).

See the [model page](https://huggingface.co/joeddav/xlm-roberta-large-xnli) for more.

### Different Pipeline models

[Read here](https://huggingface.co/docs/transformers/main_classes/pipelines) about different models available from Huggingface pipeline.

#### Text Generation

In [73]:
text_gen = pipeline("text-generation", model='gpt2') # to utilize GPU

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [62]:
prompt = "Data Science is"
text_gen(prompt, max_length=30, num_return_sequences=3)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Data Science is a subscription-based publication that is available for Kindle, iPad, Android and Apple iOS apps. Subscribe today! The Science of Technology section'},
 {'generated_text': 'Data Science is based on data by National Renewable Energy Laboratory (NREL). Each figure represents three or four years of research. Data are released after'},
 {'generated_text': 'Data Science is the largest science community in Washington D.C., and publishes over 65,500 papers daily. We support academic open science and believe that'}]

You can play around with different starting sentence. You can change `max_length` argument if you want shorter or longer sentences.

#### Sentiment Analysis

The sentiment analysis example in the beginning of the notebook can also be done using a sentiment analysis pipeline

In [63]:
## Create a new sentiment-analysis pipeline and play with the examples in the new pipeline
## HINT: You don't need to provide labels to the sentiment analysis pipeline as it is trained for the same task

In [74]:
from transformers import pipeline

# Create a sentiment analysis pipeline
sentiment_classifier = pipeline("sentiment-analysis")  # no labels needed

# Try a few example sentences
examples = [
    "I absolutely love this NLP course!",
    "This is the worst restaurant I have ever been to.",
    "The movie was okay, nothing special but not terrible either."
]

for text in examples:
    result = sentiment_classifier(text)[0]
    print(f"Text: {text}")
    print(f"Predicted label: {result['label']}, score: {result['score']:.4f}")
    print("-" * 60)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Text: I absolutely love this NLP course!
Predicted label: POSITIVE, score: 0.9999
------------------------------------------------------------
Text: This is the worst restaurant I have ever been to.
Predicted label: NEGATIVE, score: 0.9998
------------------------------------------------------------
Text: The movie was okay, nothing special but not terrible either.
Predicted label: NEGATIVE, score: 0.9509
------------------------------------------------------------


Now we use a pre‑trained sentiment‑analysis pipeline.
Unlike the zero‑shot pipeline, we do not provide labels – the model is already trained to output sentiment labels (e.g. POSITIVE / NEGATIVE).

### OPTIONAL
#### You can create a Hugging face account and create a token if you wish to create or push content to a repository (e.g., when training a model or modifying a model card) within hugging face.

- Create an account at https://huggingface.co/
- After logging in
    - go to Settings->Access Tokens
    - Create new token and give write permissions

- Run these commands 
    - `brew install huggingface-cli`
    - `huggingface-cli login` and paste the access token from huggingface
    - **Do not add access token for github if it asks**

    Reference: https://huggingface.co/docs/hub/security-tokens

What did we do and learn in this notebook:
* demonstrating extended use of Hugging Face’s zero-shot-classification pipeline.
Show how to classify custom sentences with user‑defined labels, and present predictions in a tabular format via a helper function.
* Explore robustness of zero‑shot models with small spelling changes and different candidate label sets.
* Use a multilingual NLI model (joeddav/xlm-roberta-large-xnli) to classify non‑English text (e.g. Russian) into English topic labels.
* Introduce a separate sentiment-analysis pipeline to contrast zero‑shot classification with a task‑specific pretrained model (no custom labels required).